# ReAct Agent + RAG Tool

In this notebook we will build a [ReAct](https://react-lm.github.io/) agent capable of answering questions about specific source information/documents. This agent will use a technique known as [Retrieval Augmented Generation (RAG)](https://python.langchain.com/docs/concepts/rag/).

Retrieval Augmented Generation (RAG) is a powerful technique that enhances language models by combining them with external knowledge bases. RAG addresses a key limitation of models: models rely on fixed training datasets, which can lead to outdated or incomplete information.

When given a query:
1. RAG systems first search a knowledge base for relevant information
2. The system then incorporates this retrieved information into the model's prompt
3. The model uses the provided context to generate a response to the query.

By bridging the gap between vast language models and dynamic, targeted information retrieval, RAG is a powerful technique for building more capable and reliable AI systems.

A typical RAG application has two main components:
1. **Ingestion** or **Indexing**: a pipeline for ingesting data from a source and indexing it. *This usually happens offline.*
2. **Retrieval and generation**: the actual RAG chain, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

In [1]:
# load the environment variables
import os
from dotenv import load_dotenv
load_dotenv(verbose=True)

True

## 1. Ingestion (or Indexing)
The ingestion/indexing process follow these steps:
1. Load: First we need to load our data. Since we are working with a PDF file, we will use [LangChain's PyPDFLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html) to parse the PDF file content.
    >If you want to load other file types, you can explore [LangChain's repository of document loaders](https://python.langchain.com/docs/integrations/document_loaders/) or simply use [LangChain's DirectoryLoader](https://python.langchain.com/docs/how_to/document_loader_directory/), which is a simple interface that allows us to load a range of file types out-of-the-box.
2. Split: Text splitters break large Documents into smaller chunks. This is useful both for indexing data and passing it into a model, as large chunks are harder to search over and won't fit in a model's finite context window. We will use [LangChain's RecursiveCharacterTextSplitter](https://python.langchain.com/docs/concepts/text_splitters/#text-structured-based) to split the documents.
3. Embed and Store: We need somewhere to store and index our splits, so that they can be searched over later. This is often done using a VectorStore and Embeddings model. In this notebook, we will use [Chroma](https://github.com/chroma-core/chroma), a simple and easy-to-use open-source embedding database.

![indexing-steps](https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png)

The complete implementations of this process can be found in `src/examples/ingestion/ingest_data.py`

### Load Documents

In this example, we'll load and ingest [Meta's Terms of Service](https://mbasic.facebook.com/legal/terms/plain_text_terms/) so that we can ask questions and better understand a document most of us have probably agreed to but never actually read!

In [2]:
from langchain_community.document_loaders import PyPDFLoader

data_directory = "../data/"
file_names = os.listdir(data_directory)  # get files from data directory

# create a list to store all pages from all documents
docs_pages = []

for i, file_name in enumerate(file_names, start=1):
    if file_name.lower().endswith(".pdf"):
        print(f"[{i}/{len(file_names)}] Loading {file_name} ... ", end="")
        loader = PyPDFLoader(
            file_path=os.path.join(data_directory, file_name),
            mode="page",
            extract_images=False,
            extraction_mode="plain",
        )
        async for page in loader.alazy_load():
            docs_pages.append(page)
        print(f"Loaded {len(docs_pages)} pages from {file_name}")

print(f"Loaded {len(docs_pages)} pages from {len(file_names)} PDF document(s)")

[1/1] Loading Meta Terms of Service.pdf ... Loaded 12 pages from Meta Terms of Service.pdf
Loaded 12 pages from 1 PDF document(s)


In [3]:
docs_pages[0]

Document(metadata={'producer': 'Skia/PDF m137', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36', 'creationdate': '2025-06-23T21:10:36+00:00', 'title': 'Meta Terms of Service', 'moddate': '2025-06-23T21:10:36+00:00', 'source': '../data/Meta Terms of Service.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='Terms of Service\nExplore the policy\nOverview\n1. The services we provide\n2. How our services are funded\n3. Your commitments to Facebook and our community\n4. Additional provisions\n5. Other terms and policies that may apply to you\nOverview\nEffective January 1, 2025\nMeta builds technologies and services that enable people to connect with each oth‐\ner, build communities, and grow businesses. These Terms of Service (the "Terms")\ngovern your access and use of Facebook, Messenger, and the other products, web‐\nsites, features, apps, services, technologies, and software we offer (

### Split Documents

In [4]:
# filter unnecessary metadata from the loaded documents
metadata_to_remove = ["producer", "creator", "creationdate", "moddate"]

for page in docs_pages:
    # remove unnecessary metadata
    for metadata_key in metadata_to_remove:
        page.metadata.pop(metadata_key, None)
    # make page numbers start at 1 (PyPDFLoader indexes pages from 0)
    if "page" in page.metadata and isinstance(page.metadata["page"], int):
        page.metadata["page"] += 1

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# split loaded pages into chunks of 1000 characters with 200 characters of overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
doc_splits = text_splitter.split_documents(docs_pages)

print(f"Split {len(docs_pages)} documents into {len(doc_splits)} chunks")

Split 12 documents into 44 chunks


In [6]:
doc_splits[0]

Document(metadata={'title': 'Meta Terms of Service', 'source': '../data/Meta Terms of Service.pdf', 'total_pages': 12, 'page': 1, 'page_label': '1'}, page_content='Terms of Service\nExplore the policy\nOverview\n1. The services we provide\n2. How our services are funded\n3. Your commitments to Facebook and our community\n4. Additional provisions\n5. Other terms and policies that may apply to you\nOverview\nEffective January 1, 2025\nMeta builds technologies and services that enable people to connect with each oth‐\ner, build communities, and grow businesses. These Terms of Service (the "Terms")\ngovern your access and use of Facebook, Messenger, and the other products, web‐\nsites, features, apps, services, technologies, and software we offer (the Meta\nProducts or Products), except where we expressly state that separate terms (and\nnot these) apply. (For example, your use of Instagram is subject to the Instagram\nTerms of Use). These Products are provided to you by Meta Platforms, Inc

### Embed and Store Documents

In [7]:
from chromadb import PersistentClient
from chromadb.config import Settings
from examples.config import embedding_model
from langchain_chroma import Chroma

vector_store_directory = "../vector_store/"
collection_name = "meta_terms_of_service"

vector_store_client = PersistentClient(
    path=vector_store_directory,
    settings=Settings(anonymized_telemetry=False)
)

vector_store = Chroma(
    client=vector_store_client,
    collection_name=collection_name,
    collection_metadata={"num_files": len(file_names), "file_names": ", ".join(file_names)},
    embedding_function=embedding_model,
)

2025-07-12 18:52:09,944 - INFO - examples.config - Using OpenAI model: gpt-4.1
2025-07-12 18:52:09,945 - INFO - examples.config - Using OpenAI embedding model: text-embedding-3-small


In [8]:
# Index chunks
chunk_indexes = vector_store.add_documents(documents=doc_splits)

2025-07-12 18:52:12,132 - INFO - httpx - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## 2. Retrieval and generation
1. Retrieve: Given a user input, relevant splits are retrieved from storage using a [Retriever](https://python.langchain.com/docs/concepts/retrievers/).
2. Generate: A ChatModel / LLM produces an answer using a prompt that includes both the question with the retrieved data

![retrieval-and-generation](https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)

To do this, we will use the ReAct agent associated with a RAG tool.

In [9]:
vector_store._collection_metadata

{'num_files': 1, 'file_names': 'Meta Terms of Service.pdf'}

In [10]:
from langchain_core.tools import tool
from textwrap import dedent

retrieval_tool_description = f"""\
Search and retrieve information from documents to answer a user query.
You have access to the following {vector_store._collection_metadata["num_files"]} document(s):
{vector_store._collection_metadata["file_names"]}
"""

@tool(response_format="content_and_artifact", description=retrieval_tool_description)
def retrieve(query: str):
    # retrieve documents from the vector store with max marginal relevance
    retrieved_chunks = vector_store.max_marginal_relevance_search(query, k=3)
    # format the retrieved chunks into a single string
    context = "\n\n".join(
        (
            f"## {i}. Retrieved Document Chunk\n\n"
            f"### Chunk Metadata:\n{doc.metadata}\n\n"
            f"### Chunk Content:\n{doc.page_content}"
        ) for i, doc in enumerate(retrieved_chunks, start=1)
    )
    # build message with the context to be used by the LLM
    context_message = dedent(
        """\
        Use the following pieces of context retrieved from the documents to answer the question.
        If you don't have enough information to answer the question, say that you can't answer it.

        # Context

        {context}
        """
    ).format(context=context)
    
    return context_message, retrieved_chunks

In [11]:
from examples.agents.react.agent import ReActAgent
from examples.config import llm

react_rag_agent = ReActAgent(
    llm=llm,
    tools=[retrieve],
    system_prompt="You are a helpful assistant for question-answering tasks.",
)

### Run ReAct Agent with RAG tool

In [12]:
from langchain_core.messages import HumanMessage, ToolMessage

# Define the input
messages = [
    HumanMessage(content="What can Meta do with my personal data?"),
]

# Run the graph
react_output = react_rag_agent.run(input={"messages": messages})

2025-07-12 18:52:25,468 - INFO - httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-12 18:52:25,996 - INFO - httpx - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-12 18:52:30,021 - INFO - httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [13]:
# get messages and tool outputs
for m in react_output["messages"]:
    m.pretty_print()
    if isinstance(m, ToolMessage):
        print()
        print(f" --> Tool artifact: {m.artifact} (type: {type(m.artifact)})")

================================ Human Message =================================

What can Meta do with my personal data?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_nfGsblcmYaKaiF31sqwowqc5)
 Call ID: call_nfGsblcmYaKaiF31sqwowqc5
  Args:
    query: What can Meta do with my personal data?
================================= Tool Message =================================
Name: retrieve

Use the following pieces of context retrieved from the documents to answer the question.
If you don't have enough information to answer the question, say that you can't answer it.

# Context

## 1. Retrieved Document Chunk

### Chunk Metadata:
{'page': 3, 'page_label': '3', 'title': 'Meta Terms of Service', 'source': '../data/Meta Terms of Service.pdf', 'total_pages': 12}

### Chunk Content:
entities and develop advanced technical systems to detect potential misuse of our
Products, harmful conduct towards others, and situations where we may

In [14]:
final_message = react_output["messages"][-1]
print(final_message.content)

Meta can do the following with your personal data, according to their Terms of Service:

1. Personalized Advertising: Meta uses your personal data, such as your activity and interests, to show you personalized ads and sponsored content that they believe will be more relevant to you. Advertisers can specify what audience they want to reach, but Meta states that it does not sell your personal data to advertisers. Instead, it uses your information to match you with relevant ads without directly telling advertisers who you are.

2. Sharing Data for Safety and Security: Meta may access, preserve, use, and share information it collects about you with other Meta companies, especially if there is potential misuse of their products, harmful conduct, or to help keep users and the community safe. This includes responding to reports of violating content, taking actions like removing content or disabling accounts, and sometimes contacting law enforcement.

3. Advertiser Reports: Meta provides adver

### List Ingested Collections

In [15]:
from examples.ingestion import list_collections_and_folders

list_collections_and_folders()

2025-07-12 18:52:48,049 - INFO - examples.ingestion.list_data -   Listing collections and their segment folders...
2025-07-12 18:52:48,051 - INFO - examples.ingestion.list_data -   Found 2 collections:
2025-07-12 18:52:48,052 - INFO - examples.ingestion.list_data -   Collection: 'meta_terms_of_service' -> ID: 635f6d73-bdb1-43eb-935b-e51d3a8d835d | metadata: {'num_files': 1, 'file_names': 'Meta Terms of Service.pdf'}
2025-07-12 18:52:48,053 - INFO - examples.ingestion.list_data -   Found VECTOR segment: da866460-8f85-48a7-8048-564d2ebabcb9 (urn:chroma:segment/vector/hnsw-local-persisted) - has folder
2025-07-12 18:52:48,053 - INFO - examples.ingestion.list_data -   Found METADATA segment: 284beb04-7016-4039-937e-4424752d904d (urn:chroma:segment/metadata/sqlite) - no folder
2025-07-12 18:52:48,054 - INFO - examples.ingestion.list_data -   Collection has 1 VECTOR segment(s) and 1 METADATA segment(s)
2025-07-12 18:52:48,055 - INFO - examples.ingestion.list_data -      VECTOR segment folder

### Delete Ingested Collection

In [16]:
from examples.ingestion import delete_collection_data

print(f"Deleting ingested data from collection '{collection_name}'...")

delete_collection_data(collection_name)

2025-07-12 18:54:44,169 - INFO - examples.ingestion.delete_data - Deleting data for collection 'meta_terms_of_service' from the vector store ...
2025-07-12 18:54:44,170 - INFO - examples.ingestion.delete_data - Found collection ID: 635f6d73-bdb1-43eb-935b-e51d3a8d835d
2025-07-12 18:54:44,171 - INFO - examples.ingestion.delete_data - Found VECTOR segment: da866460-8f85-48a7-8048-564d2ebabcb9 (urn:chroma:segment/vector/hnsw-local-persisted) - has folder
2025-07-12 18:54:44,171 - INFO - examples.ingestion.delete_data - Found METADATA segment: 284beb04-7016-4039-937e-4424752d904d (urn:chroma:segment/metadata/sqlite) - no folder
2025-07-12 18:54:44,171 - INFO - examples.ingestion.delete_data - Will delete 1 VECTOR segment folder(s), skipping 1 METADATA segment(s)
2025-07-12 18:54:44,185 - INFO - examples.ingestion.delete_data - Collection 'meta_terms_of_service' deleted from ChromaDB client.
2025-07-12 18:54:44,187 - INFO - examples.ingestion.delete_data - Removing segment folder: da866460-

Deleting ingested data from collection 'meta_terms_of_service'...
